In [1]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
from maritime_traffic_network import MaritimeTrafficNetwork
import folium
import warnings
import sys
import pickle
warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

Geopandas has version 0.13.2
Movingpandas has version 0.17.1


In [2]:
# add paths for modules
sys.path.append('../visualization')
sys.path.append('../features')
print(sys.path)

# import modules
import visualize

['/Users/janhendrikwebert/maritime_route_prediction/src/models', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python311.zip', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11/lib-dynload', '', '/Users/janhendrikwebert/miniforge3/envs/env_geo/lib/python3.11/site-packages', '../visualization', '../features', '../models', '../features', '../visualization', '../features']


In [3]:
# read data from file
datasize = 'full'
location = 'stavanger'
data_dates = ['202204']
filenames = []

for i in range(0, len(data_dates)):
    data_date = data_dates[i]
    # load path data from file
    filename = '../../data/processed/'+data_date+'_points_'+location+'_cleaned_meta_'+datasize+'_dualSplit_3.parquet'
    filenames.append(filename)
    gdf_part = gpd.read_parquet(filename)

    if i==0:
        gdf = gdf_part
    else:
        gdf = pd.concat([gdf, gdf_part])

data_date = str(data_dates)
filename = str(filenames)

In [4]:
# filter the data according to ship type
#shiptypes = ['Tank', 'Last', 'Passasjer', None]
#gdf = gdf[gdf.skipsgruppe.isin(shiptypes)]

In [5]:
# Transform to desired CRS
# 4326 for WGS 84 (global) // 32632 for UTM 32N (Norway)
crs = 32632  # Coordinate reference system
gdf.to_crs(crs, inplace=True)  # Transformation
gdf.head()

,mmsi,imo_nr,length,lon,lat,sog,cog,true_heading,nav_status,message_nr,bredde,dypgaaende,skipstype,skipsgruppe,fartoynavn,geometry,speed
date_time_utc,,,,,,,,,,,,,,,,,
2022-04-08 03:12:48,205209000_0_2022-04-08 03:12:48,0,51,5.36673,59.6732,10.6,225.9,225,0,1,NaN,NaN,None,None,None,POINT (295416.506 6620618.919),11.173331
2022-04-08 03:12:58,205209000_0_2022-04-08 03:12:48,0,51,5.36531,59.6725,10.7,227.2,228,0,1,NaN,NaN,None,None,None,POINT (295332.330 6620545.422),11.173331
2022-04-08 03:13:08,205209000_0_2022-04-08 03:12:48,0,51,5.36458,59.6722,10.6,228.8,230,0,1,NaN,NaN,None,None,None,POINT (295289.421 6620514.299),5.300198
2022-04-08 03:13:27,205209000_0_2022-04-08 03:12:48,0,51,5.36385,59.6718,10.5,229.4,229,0,1,NaN,NaN,None,None,None,POINT (295245.901 6620472.052),3.191942
2022-04-08 03:13:38,205209000_0_2022-04-08 03:12:48,0,51,5.36313,59.6715,10.5,225.2,223,0,1,NaN,NaN,None,None,None,POINT (295203.553 6620440.899),4.778771


In [6]:
# initialize maritime traffic network
network = MaritimeTrafficNetwork(gdf, crs)
network.get_trajectories_info()

Number of AIS messages: 5678853
Number of trajectories: 19023
Coordinate Reference System (CRS): EPSG:32632


In [ ]:
# calculate significant turning points using Douglas Peucker algorithm
tolerance = 10 # DP tolerance parameter 0.0002
network.calc_significant_points_DP(tolerance)

Calculating significant turning points with Douglas Peucker algorithm (tolerance = 10) ...
Number of significant points detected: 427022 (7.52% of AIS messages)
Time elapsed: 4.73 minutes
Adding course over ground before and after each turn ...


In [ ]:
# detect waypoints using spatial clustering
method = 'DBSCAN'      # 'DBSCAN' , 'HDBSCAN', 'OPTICS'
metric = 'mahalanobis'  # 'euclidean', 'mahalanobis', 'haversine'
min_samples = 10
min_cluster_size = 10
eps = 50
V = np.diag([1, 1, 0.01, 0.01, 1])  # mahalanobis distance parameter matrix V = np.diag([1, 1, 0.01, 0.01, 1e-3])  seems to be good
network.calc_waypoints_clustering(method=method, min_samples=min_samples, min_cluster_size=min_cluster_size,
                                  eps=eps, metric=metric, V=V)

In [ ]:
# make graph from waypoints
max_distance=20
max_angle=45
network.make_graph_from_waypoints(max_distance=max_distance, max_angle=max_angle)

In [ ]:
model = data_date+'_waypoints_DP' + str(tolerance) + '_' + method + str(min_samples) +'_'+location+'_'+datasize+'_UTM_filtered'
merge_stops = True
merge_stops_speed = 2
pruning = 1
network.merge_stop_points(max_speed=merge_stops_speed)
network.prune_graph(pruning)

In [ ]:
# save hyperparameters
params = {
    'Data':filename,
    'DP_tolerance':tolerance,
    'clustering_method':method,
    'clustering_metric':metric,
    'clustering_min_samples':min_samples,
    'clustering_min_cluster_size':min_cluster_size,
    'clustering_eps':eps,
    'clustering_metric_V':V,
    'graph_generation_max_distance':max_distance,
    'graph_generation_max_angle':max_angle,
    'merge_stops_speed':merge_stops_speed
}
network.set_hyperparameters(params)

In [ ]:
# save network as pickle object
fileObj = open('../../data/interim/'+data_date+'_waypoints_DP'+str(tolerance)+'_'+method+str(min_samples)+'_'+location+'_'+datasize+'_UTM.obj', 'wb')
pickle.dump(network, fileObj)
fileObj.close()